#### This particular notebook includes downstream analyses and comparisons between:
* Dataset 1: Xenium (rep 1 only) vs CosMx
* Dataset 1: Xenium rep 1 vs rep 2
* Dataset 1: Xenium Int reps 1 and 2 -- no comparison, just an examination

It can be adapted to compare any 2 datasets or reps/slides within a dataset.

#### Required input files:
* Annotated cell-based data object (for each dataset)

Environment: Please create and activate the conda environment provided in default_env.yaml before running this notebook

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scanpy as sc
import squidpy as sq

import gzip
import anndata

import os

## CosMx and Xenium Downstream Analysis Plots -- Annotation-Dependent

In [ ]:
Int_XeniumData = sc.read_h5ad('/path/25_11_22_Xenium_Dataset1_290_IntReps1and2_Annotated.h5ad')

CosMxData = sc.read_h5ad('/path/25_11_22_CosMx_Annotated.h5ad')

In [ ]:
## Subset Int_XeniumData to create a Rep1 only version

Rep1_XeniumData = Int_XeniumData[Int_XeniumData.obs['batch'] == 'Sept'].copy()

Rep1_XeniumData.obs

In [ ]:
# Set output directory

sc.settings.figdir = '/path/'

In [ ]:
## Sanity check

display(Int_XeniumData.obs)
display(CosMxData.obs)

#### UMAPs

In [ ]:
# Create a color map dictionary
Rep1XeniumCoarse_color_map = {
    'B': 'darkorange',
    'Cytotoxic': 'red',
    'Endothelial': 'lime',
    'Enteric_glia': 'green',
    'Fibroblast': 'navy',
    'IEC': 'saddlebrown',
    'MAST': 'gold',
    'MNP': 'deepskyblue',
    'Myofibroblast': 'cadetblue',
    'Neutrophil': 'darkviolet',
    'Pericyte': 'tan', 
    'Plasma': 'lightpink',
    'T': 'mediumvioletred',
#    'Unassigned': 'lightgray',
#    'ncount_hi': 'black',
}

# Ensure the column is categorical
Rep1_XeniumData.obs['25_06_11_Common_Coarse_Xenium_Combinedv3'] = pd.Categorical(
    Rep1_XeniumData.obs['25_06_11_Common_Coarse_Xenium_Combinedv3'],
    categories=Rep1XeniumCoarse_color_map.keys(),  # This sets the order of categories based on color map keys
    ordered=True
)

# Plot UMAP with the custom color palette
sc.pl.umap(Rep1_XeniumData, 
           color='25_06_11_Common_Coarse_Xenium_Combinedv3', 
           palette=list(Rep1XeniumCoarse_color_map.values()),
           wspace=0.4,
         #  save = '_Rep1_Xenium_CommonCoarseAnnotations_Colormap.pdf'
          )

In [ ]:
sc.pl.umap(
    Rep1_XeniumData,
    color=[
        "24_05_29_Fine_annotations_Xenium_combined",
    ],
    wspace=0.4,
  #  save = '_Rep1_Xenium_FineAnnotations.pdf',
)

In [ ]:
sc.pl.umap(
    Int_XeniumData,
    color=[
        "24_05_29_Fine_annotations_Xenium_combined",
    ],
    wspace=0.4,
  #  save = '_Int_Xenium_FineAnnotations.pdf',
)

In [ ]:
# Create a color map dictionary
CosMxCoarse_color_map = {
#    'B': 'darkorange',
#    'Cytotoxic': 'red',
    'Endothelial': 'lime',
#    'Enteric_glia': 'green',
    'Fibroblast': 'navy',
    'IEC': 'saddlebrown',
#    'MAST': 'gold',
    'MNP': 'deepskyblue',
    'Myofibroblast': 'cadetblue',
#    'Neutrophil': 'darkviolet',
    'Plasma': 'lightpink',
#    'T': 'mediumvioletred',
    'Unassigned': 'lightgray',
    'ncount_hi': 'black',
}

# Ensure the column is categorical
CosMxData.obs['24_02_27_Common_coarse_newCosMxobj'] = pd.Categorical(
    CosMxData.obs['24_02_27_Common_coarse_newCosMxobj'],
    categories=CosMxCoarse_color_map.keys(),  # This sets the order of categories based on color map keys
    ordered=True
)

# Plot UMAP with the custom color palette
sc.pl.umap(CosMxData, 
           color='24_02_27_Common_coarse_newCosMxobj', 
           palette=list(CosMxCoarse_color_map.values()),
           wspace=0.4,
         #  save = '_CosMx_CommonCoarseAnnotations_Colormap.pdf'
          )

In [ ]:
## Make new UMAP coloring for CosMx Fine annotations

# Create a color map dictionary
CosMxFine_color_map = {
    'Endothelial': 'blue',
    'Fibroblast': 'orange',
    'Fibroblast_CCL19_CCL21_hi': 'green',
    'IEC_CT_colonocyte': 'red',
    'IEC_colonocyte': 'darkviolet',
    'IEC_crypt_base': 'brown',
    'MNP': 'pink',
    'Myofibroblast': 'yellowgreen',
    'Plasma': 'cyan',
    'Plasma_IgG': 'skyblue',
    'Plasma_IgM_IgA': 'deeppink',
    'Unassigned': 'lightgray',
    'ncount_hi': 'black',
}

# Ensure the column is categorical
CosMxData.obs['24_02_28_Fine_annotations_newCosMxobj_from_leiden'] = pd.Categorical(
    CosMxData.obs['24_02_28_Fine_annotations_newCosMxobj_from_leiden'],
    categories=CosMxFine_color_map.keys(),  # This sets the order of categories based on color map keys
    ordered=True
)

# Plot UMAP with the custom color palette
sc.pl.umap(CosMxData, 
           color='24_02_28_Fine_annotations_newCosMxobj_from_leiden', 
           palette=list(CosMxFine_color_map.values()),
           wspace=0.4,
         #  save = '_CosMx_FineAnnotations_Colormap.pdf'
          )

#### Marker gene visualization

Rep1_Xenium

In [ ]:
## Visualize marker genes

Rep1_XeniumData.uns['log1p']["base"] = None

# Coarse annotations

sc.tl.rank_genes_groups(Rep1_XeniumData, layer='log_normalized_counts', groupby='25_06_11_Common_Coarse_Xenium_Combinedv3', method='wilcoxon')
sc.pl.rank_genes_groups(Rep1_XeniumData, n_genes=25, sharey=False)

## Compute dendrogram

# Coarse annotations

sc.tl.dendrogram(Rep1_XeniumData, '25_06_11_Common_Coarse_Xenium_Combinedv3')

ax = sc.pl.dendrogram(Rep1_XeniumData, '25_06_11_Common_Coarse_Xenium_Combinedv3')

In [ ]:
## Matrixplot (heatmap)

# Set font size for all plots (specify for individual plots if needed)
sc.set_figure_params(scanpy=True, fontsize=11)

sc.pl.rank_genes_groups_matrixplot(Rep1_XeniumData, n_genes=5, layer='log_normalized_counts', groupby = '25_06_11_Common_Coarse_Xenium_Combinedv3', vmin=0, vmax=2, cmap='Reds', colorbar_title='Mean gene expression', figsize=(13,5)) #, save='_rgg_Rep1_Xenium_MeanGeneExp_LogNormCounts_CommonCoarseAnnotations.pdf')

# Title: 'Mean gene expression levels per cluster, using log normalized counts'
# Using automatically generated top DE gene list
# Using Coarse annotations

In [ ]:
ax = sc.pl.correlation_matrix(Rep1_XeniumData, '25_06_11_Common_Coarse_Xenium_Combinedv3', figsize=(6.25,4.5)) #, save='_Rep1_Xenium_correlationmatrix_CommonCoarseAnnotations.pdf')

# Zooming out to cell types instead of specific genes
# Using Coarse annotations

CosMx

In [ ]:
## Visualize marker genes

CosMxData.uns['log1p']["base"] = None

# Coarse annotations

sc.tl.rank_genes_groups(CosMxData, layer='log_normalized_counts', groupby='24_02_27_Common_coarse_newCosMxobj', method='wilcoxon')
sc.pl.rank_genes_groups(CosMxData, n_genes=25, sharey=False)

## Compute dendrogram

# Coarse annotations

sc.tl.dendrogram(CosMxData, '24_02_27_Common_coarse_newCosMxobj')

ax = sc.pl.dendrogram(CosMxData, '24_02_27_Common_coarse_newCosMxobj')

In [ ]:
## Matrixplot (heatmap)

# Set font size for all plots (specify for individual plots if needed)
sc.set_figure_params(scanpy=True, fontsize=11)

sc.pl.rank_genes_groups_matrixplot(CosMxData, n_genes=5, layer='log_normalized_counts', groupby = '24_02_27_Common_coarse_newCosMxobj', vmin=0, vmax=2, cmap='Reds', colorbar_title='Mean gene expression', figsize=(13,5)) #, save='_rgg_CosMx_MeanGeneExp_LogNormCounts_CommonCoarseAnnotations.pdf')

# Title: 'Mean gene expression levels per cluster, using log normalized counts'
# Using automatically generated top DE gene list
# Using Coarse annotations

In [ ]:
ax = sc.pl.correlation_matrix(CosMxData, '24_02_27_Common_coarse_newCosMxobj', figsize=(6.25,4.5)) #, save='_CosMx_correlationmatrix_CommonCoarseAnnotations.pdf')

# Zooming out to cell types instead of specific genes
# Using Coarse annotations

#### Cell type quantification plots

Int_XeniumData

In [ ]:
# Group by annotations and batch, then count the occurrences
grouped_counts = Int_XeniumData.obs.groupby(['24_05_29_Fine_annotations_Xenium_combined', 'batch']).size().unstack(fill_value=0)

# Calculate the percentages
# First sum is summing along columns (summing counts per batch), second sum is summing these totals
total_count = grouped_counts.sum().sum() 
percentages = (grouped_counts / total_count * 100).reset_index()

# Calculate combined percentages for each annotation for sorting
# This sums the values across columns (the Rep1 and Rep2 batches) to find combined percentage of total dataset
combined_percentages = percentages.set_index('24_05_29_Fine_annotations_Xenium_combined').sum(axis=1)

# Sort annotations by combined percentages from greatest to least
sorted_annotations = combined_percentages.sort_values(ascending=False).index

# Reorder the percentages DataFrame according to sorted_annotations
percentages['sort_order'] = pd.Categorical(percentages['24_05_29_Fine_annotations_Xenium_combined'], categories=sorted_annotations, ordered=True)
percentages.sort_values('sort_order', inplace=True)

# View
percentages

In [ ]:
# Plot

plt.figure(figsize=(12, 8))

# Define colors for each batch
batch_colors = {'Sept': 'forestgreen', 'Nov': 'darkviolet'}

# Define legend labels for each batch
batch_legend_labels = {'Sept': 'Rep 1', 'Nov': 'Rep 2'}

# Assuming 'percentages' and 'sorted_annotations' are already defined
annotations = sorted_annotations
batches = percentages.columns[1:-1]  # Exclude non-batch columns

# Initialize the bottom for each annotation's bar start (bottom of the stack)
bottoms = np.zeros(len(annotations))

# Plot each batch as a part of the stack
for batch in batches:
    batch_percentages = []
    for annotation in annotations:
        percentage = percentages.loc[percentages['24_05_29_Fine_annotations_Xenium_combined'] == annotation, batch]
        batch_percentages.append(percentage.values[0] if not percentage.empty else 0)
    
    plt.bar(annotations, batch_percentages, bottom=bottoms, label=batch_legend_labels.get(batch, batch), color=batch_colors.get(batch, 'grey'))
    bottoms += np.array(batch_percentages)

# Customize plot appearance
plt.xlabel('Fine Annotations', fontsize=12)
plt.ylabel('Percentage of Total', fontsize=12)
plt.title('Integrated Xenium Dataset: Cell Type Quantification by Replicate', fontsize=13)
plt.xticks(rotation=45, ha='right', fontsize=12)
plt.yticks(fontsize=12)

# Make the legend bigger
plt.legend(title='Replicate', fontsize=12, title_fontsize='12')

# Get rid of the grid lines
plt.grid(False)

# Set layout
plt.tight_layout()

# Save figure
#plt.savefig('Int_Xenium_celltype_quantification_FineAnnotations.pdf', bbox_inches='tight')

# Display
plt.show()


Rep1_XeniumData and CosMxData

In [ ]:
## Compare the coarse annotation cell type quantification

# Initial calculation of percentages
SXD_counts = Rep1_XeniumData.obs['25_06_11_Common_Coarse_Xenium_Combinedv3'].value_counts(normalize=True) * 100
CD_counts = CosMxData.obs['24_02_27_Common_coarse_newCosMxobj'].value_counts(normalize=True) * 100

# Ensure both Series have a matching index for annotations
annotations = SXD_counts.index.union(CD_counts.index)
SXD_counts = SXD_counts.reindex(annotations, fill_value=0)
CD_counts = CD_counts.reindex(annotations, fill_value=0)

# Create a DataFrame with 'Xenium' and 'CosMx' as columns directly, and annotations as rows
comparison_df = pd.DataFrame({
    'Xenium': SXD_counts,
    'CosMx': CD_counts
}, index=annotations)

# Define color map (matches umaps)
CombinedCoarse_color_map = {
    'B': 'darkorange',
    'Cytotoxic': 'red',
    'Endothelial': 'lime',
    'Enteric_glia': 'green',
    'Fibroblast': 'navy',
    'IEC': 'saddlebrown',
    'MAST': 'gold',
    'MNP': 'deepskyblue',
    'Myofibroblast': 'cadetblue',
    'Neutrophil': 'darkviolet',
    'Pericyte': 'tan', # new
    'Plasma': 'lightpink',
    'T': 'mediumvioletred',
    'Unassigned': 'lightgray',
    'ncount_hi': 'black',
}

# Create a DataFrame with 'Xenium' and 'CosMx' as columns directly, and annotations as rows
comparison_df = pd.DataFrame({
    'Xenium': SXD_counts,
    'CosMx': CD_counts
}, index=annotations)

# Plot
fig, ax = plt.subplots(figsize=(8, 6))

datasets = ['Xenium', 'CosMx']
colors = [CombinedCoarse_color_map[annotation] for annotation in annotations]

# Base height for the stacked bars
bottom = np.zeros(2)

# Iterate through each annotation and stack them
for annotation, color in zip(annotations, colors):
    heights = comparison_df.loc[annotation, datasets]
    ax.bar(datasets, heights, bottom=bottom, label=annotation, color=color)
    bottom += heights

# Extend the y-axis range up to 105
ax.set_ylim(0, 105)

# Customize plot appearance
plt.xlabel('Dataset', fontsize=12)
plt.ylabel('Percentage of Total', fontsize=12)
plt.title('Cell Type Quantification by Coarse Annotations', fontsize=13)
plt.legend(title='Coarse Annotations', bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=12, title_fontsize='12')
plt.xticks(rotation=0, fontsize=12)  # Rotate x-axis labels to horizontal for readability

# Customize y-axis ticks
plt.yticks(ticks=[i for i in range(0, 101, 10)], labels=[str(i) if i % 20 == 0 else '' for i in range(0, 101, 10)], fontsize=12)

# Get rid of the grid lines
plt.grid(False)

# Set layout
plt.tight_layout()

# Save figure
#plt.savefig('/path/CosMx_vs_Xenium_celltype_quantification_CoarseAnnotations.pdf', bbox_inches='tight')

# Display
plt.show()

# Xenium Int marker gene visualization -- With fine annotations

In [ ]:
Int_XeniumData

In [ ]:
## Visualize marker genes

Int_XeniumData.uns['log1p']["base"] = None

# Coarse annotations

sc.tl.rank_genes_groups(Int_XeniumData, layer='log_normalized_counts', groupby='24_05_29_Fine_annotations_Xenium_combined', method='wilcoxon')
sc.pl.rank_genes_groups(Int_XeniumData, n_genes=25, sharey=False)

## Compute dendrogram

# Coarse annotations

sc.tl.dendrogram(Int_XeniumData, '24_05_29_Fine_annotations_Xenium_combined')

ax = sc.pl.dendrogram(Int_XeniumData, '24_05_29_Fine_annotations_Xenium_combined')

In [ ]:
## Matrixplot (heatmap) -- with 3 genes instead of 5

# Set font size for all plots (specify for individual plots if needed)
sc.set_figure_params(scanpy=True, fontsize=11)

sc.pl.rank_genes_groups_matrixplot(Int_XeniumData, n_genes=3, layer='log_normalized_counts', groupby = '24_05_29_Fine_annotations_Xenium_combined', vmin=0, vmax=2, cmap='Reds', colorbar_title='Mean gene expression', figsize=(26,10)) #, save='_rgg_Int_Xenium290_MeanGeneExp_LogNormCounts_FineAnnotations_3genes.pdf')

# Title: 'Mean gene expression levels per cluster, using log normalized counts'
# Using automatically generated top DE gene list
# Using Fine annotations

In [ ]:
ax = sc.pl.correlation_matrix(Int_XeniumData, '24_05_29_Fine_annotations_Xenium_combined', figsize=(12.5,9)) #, save='_Int_Xenium290_correlationmatrix_FineAnnotations.pdf')

# Zooming out to cell types instead of specific genes
# Using Fine annotations

# Additional plots for Int Xenium dataset

## DEG dot plot with coarse annotations

In [ ]:
## Visualize marker genes

Int_XeniumData.uns['log1p']["base"] = None

# Coarse annotations

sc.tl.rank_genes_groups(Int_XeniumData, layer='log_normalized_counts', groupby='25_06_11_Common_Coarse_Xenium_Combinedv3', method='wilcoxon')
sc.pl.rank_genes_groups(Int_XeniumData, n_genes=25, sharey=False)

## Compute dendrogram

# Coarse annotations

sc.tl.dendrogram(Int_XeniumData, '25_06_11_Common_Coarse_Xenium_Combinedv3')

ax = sc.pl.dendrogram(Int_XeniumData, '25_06_11_Common_Coarse_Xenium_Combinedv3')

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    Int_XeniumData,
    var_names=['C15orf48', 'EPCAM', 'AQP8', 'FABP1', 'FABP2', 'PHGR1', 'S100A8', 'S100A9', 'CD1C', 'CD19', 'LTB', 'SELL', 'MS4A1'],
    values_to_plot="logfoldchanges",
    cmap='bwr',
    vmin=-2,
    vmax=2,
   # min_logfoldchange=1,
    figsize=(12, 4), 
    dot_max=0.5, 
    colorbar_title='log fold change',
    groupby='25_06_11_Common_Coarse_Xenium_Combinedv3',
    dendrogram=True,
    swap_axes=False,
   # save='_IntXenium290_CoarseAnnotations_DEGs_CondensedList.pdf',
    show=True)

## DEG dot plot with a mix of coarse and fine annotations

In [ ]:
### Subset anndata object to DEG cell clusters of interest

# 1) Make the helper annotation column
Fine_col = '24_05_29_Fine_annotations_Xenium_combined'
new_col  = 'Manual_DEG_Annotations'

# Copy values; collapse any label that starts with "IEC_" to "IEC"
ser = Int_XeniumData.obs[Fine_col].astype(str)
manual = ser.where(~ser.str.contains(r'^IEC_', na=False), other='IEC')

# Store as categorical (nice for plotting / grouping)
Int_XeniumData.obs[new_col] = pd.Categorical(manual)

# 2) Subset to selected cell types from the new column
keep_types = ['IEC', 'B', 'Fibroblast_FRC_S4', 'Fibroblast_IAF', 'MNP_monocyte', 'Neutrophil'] # no cDC cluster

Int_XeniumData_subsetted = Int_XeniumData[Int_XeniumData.obs[new_col].isin(keep_types)].copy()
Int_XeniumData_subsetted.obs[new_col] = Int_XeniumData_subsetted.obs[new_col].cat.remove_unused_categories()

# (Optional) quick sanity check
print("Kept cells:", Int_XeniumData_subsetted.n_obs)
print(Int_XeniumData_subsetted.obs[new_col].value_counts())

In [ ]:
Int_XeniumData_subsetted.obs

In [ ]:
## Visualize marker genes

Int_XeniumData_subsetted.uns['log1p']["base"] = None

# Manual annotations

sc.tl.rank_genes_groups(Int_XeniumData_subsetted, layer='log_normalized_counts', groupby='Manual_DEG_Annotations', method='wilcoxon')
sc.pl.rank_genes_groups(Int_XeniumData_subsetted, n_genes=25, sharey=False)

## Compute dendrogram

# Manual annotations

sc.tl.dendrogram(Int_XeniumData_subsetted, 'Manual_DEG_Annotations')

ax = sc.pl.dendrogram(Int_XeniumData_subsetted, 'Manual_DEG_Annotations')

In [ ]:
### DEG Dot Plot

obs_col = 'Manual_DEG_Annotations'
desired = ['IEC', 'Neutrophil','Fibroblast_IAF','MNP_monocyte', 'B', 'Fibroblast_FRC_S4']

Int_XeniumData_subsetted.obs[obs_col] = Int_XeniumData_subsetted.obs[obs_col].astype('category').cat.remove_unused_categories()
present = [g for g in desired if g in Int_XeniumData_subsetted.obs[obs_col].cat.categories]
Int_XeniumData_subsetted.obs[obs_col] = Int_XeniumData_subsetted.obs[obs_col].cat.reorder_categories(present, ordered=True)

sc.pl.rank_genes_groups_dotplot(
    Int_XeniumData_subsetted,
    var_names=['C15orf48', 'EPCAM', 'AQP8', 'FABP1', 'FABP2', 'PHGR1', 'MMP1', 'MMP3', 'S100A8', 'S100A9', 'CD1C', 'CD19', 'LTB', 'SELL', 'MS4A1', 'C3', 'PTGDS', 'IRF8'],
    values_to_plot="logfoldchanges",
    cmap='bwr',
    vmin=-5,
    vmax=5,
   # min_logfoldchange=1,
    figsize=(5, 8), 
    dot_max=0.75, 
    colorbar_title='log fold change',
    groupby='Manual_DEG_Annotations',
    dendrogram=False,
    swap_axes=True,
   # save='_XeniumIBD290Int_ManuallySelectedAnnotations_DEGs_CondensedList.pdf',
    show=True)

## Fibroblast dot plot

In [ ]:
# Filter data object for fibroblast clusters

Int_XeniumData_Fibroblasts = Int_XeniumData[Int_XeniumData.obs['25_06_11_Common_Coarse_Xenium_Combinedv3'] == 'Fibroblast', :]

Int_XeniumData_Fibroblasts.obs

In [ ]:
Int_XeniumData_Fibroblasts.obs['24_05_29_Fine_annotations_Xenium_combined'].unique()

In [ ]:
## Visualize marker genes

Int_XeniumData_Fibroblasts.uns['log1p']["base"] = None

# Fine annotations

sc.tl.rank_genes_groups(Int_XeniumData_Fibroblasts, layer='log_normalized_counts', groupby='24_05_29_Fine_annotations_Xenium_combined', method='wilcoxon')
sc.pl.rank_genes_groups(Int_XeniumData_Fibroblasts, n_genes=25, sharey=False)

## Compute dendrogram

# Fine annotations

sc.tl.dendrogram(Int_XeniumData_Fibroblasts, '24_05_29_Fine_annotations_Xenium_combined')

ax = sc.pl.dendrogram(Int_XeniumData_Fibroblasts, '24_05_29_Fine_annotations_Xenium_combined')

In [ ]:
sc.pl.rank_genes_groups_dotplot(
    Int_XeniumData_Fibroblasts,
    var_names=['F3','SOX6','PDGFRA', 'WNT5A', 'POSTN', 'CXCL14', # S2
               'IL1R1', 'TIMP1', 'CD44', 'IL13RA2', 'MMP1', 'MMP3', 'OSMR', 'NFKBIA', 'TNFAIP3', # IAF
               'CCL8', 'ADAMDEC1', 'APOE', 'FABP5', # S1
               'C3', 'TNFSF13B', 'IRF8', 'PTGDS',  # S4
              'GSN', 'OGN', 'CCDC80', 'C7', # S3
              ],
    values_to_plot="logfoldchanges",
    cmap='bwr',
    vmin=-2,
    vmax=2,
   # min_logfoldchange=1,
    figsize=(10, 4), 
    dot_max=0.5, 
    colorbar_title='log fold change',
    groupby='24_05_29_Fine_annotations_Xenium_combined',
    dendrogram=True,
   # swap_axes=True,
  #  save='_IntXenium290_FineAnnotations_FibroblastSubset.pdf',
    show=True)

## Monocyte and Neutrophil dot plot

In [ ]:
# Filter data object for monocyte and neut clusters

Int_XeniumData_MonosNeuts = Int_XeniumData[Int_XeniumData.obs['24_05_29_Fine_annotations_Xenium_combined'].isin(['MNP_monocyte', 'Neutrophil']), :]

Int_XeniumData_MonosNeuts.obs

In [ ]:
Int_XeniumData_MonosNeuts.obs['24_05_29_Fine_annotations_Xenium_combined'].unique()

In [ ]:
## Visualize marker genes

Int_XeniumData_MonosNeuts.uns['log1p']["base"] = None

# Fine annotations

sc.tl.rank_genes_groups(Int_XeniumData_MonosNeuts, layer='log_normalized_counts', groupby='24_05_29_Fine_annotations_Xenium_combined', method='wilcoxon')
sc.pl.rank_genes_groups(Int_XeniumData_MonosNeuts, n_genes=25, sharey=False)

## Compute dendrogram

# Fine annotations

sc.tl.dendrogram(Int_XeniumData_MonosNeuts, '24_05_29_Fine_annotations_Xenium_combined')

ax = sc.pl.dendrogram(Int_XeniumData_MonosNeuts, '24_05_29_Fine_annotations_Xenium_combined')

In [ ]:
MonoNeut_names=['HLA-DRA', 'HLA-DRB1', 'VCAN', # Monos
               'CSF3R', 'FCGR3B', 'S100A8', 'S100A9', 'TREM1', # Neuts
              ]

In [ ]:
# Now use the ordered genes in the dotplot
sc.pl.rank_genes_groups_dotplot(
    Int_XeniumData_MonosNeuts,
    var_names=MonoNeut_names,  # Pass the ordered gene list
    values_to_plot="logfoldchanges",
    cmap='bwr',
    vmin=-2,
    vmax=2,
    min_logfoldchange=1,
    figsize=(8, 3),
    dot_max=0.5,
    colorbar_title='log fold change',
    groupby='24_05_29_Fine_annotations_Xenium_combined',
    dendrogram=True,
  #  swap_axes=True,  # Put genes on the y-axis and cell types on the x-axis
 #   save='IntXenium290Data_MonosNeuts_FineAnnotations.pdf',
    show=True
)